# Sample Exam: Coffee Shops

Java June is a company that owns coffee shops in a number of locations in Europe.

The company knows that stores with more reviews typically get more new customers. This is
because new customers consider the number of reviews when picking between two shops.

They want to get more insight into what leads to more reviews.

They are also interested in whether there is a link between the number of reviews and rating.

They want a report to answer these questions.

# Task 1

Before you start your analysis, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "coffee.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| Region    | Nominal. </br> Where the store is located. One of 10 possible regions (A to J).</br> Missing values should be replaced with “Unknown”.|
| Place name | Nominal. </br>The name of the store. </br>Missing values should be replaced with “Unknown”.|
| Place type  | Nominal. </br>The type of coffee shop. One of “Coffee shop”, “Cafe”, “Espresso bar”, and “Others”. </br>Missing values should be replaced with “Unknown”. |
| Rating   | Ordinal. </br>Average rating of the store from reviews. On a 5 point scale. </br>Missing values should be replaced with 0. |
| Reviews  | Nominal. </br>The number of reviews given to the store. </br>Missing values should be replaced with the overall median number.|
| Price  | Ordinal. </br>The price range of products in the store. One of '\$', '\$\$' or '\$\$\$'. </br>Missing values should be replaced with ”Unknown”.|
| Delivery Option   | Nominal. </br>If delivery is available. Either True or False. </br>Missing values should be replaced with False. |
| Dine in Option | Nominal. </br>If dine in is available. Either True or False. </br>Missing values should be replaced with False. |
| Takeaway Option | Nominal. </br>If take away is available. Either True or False. </br>Missing values should be replaced with False.|

In [27]:
# Use this cell to write your code for task 1

import pandas as pd
import numpy as np

df = pd.read_csv('coffee.csv')

dict = {'Rating': 0,
       'Reviews': df['Reviews'].median(),
       'Dine in option': False,
       'Takeout option': False}
df.fillna(dict, inplace=True)

rating_cat = [0, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0]
price_cat = ['$', '$$', '$$$']
df['Rating'] = pd.Categorical(df['Rating'], categories=rating_cat, ordered=True)
df['Price'] = pd.Categorical(df['Price'], categories=price_cat, ordered=True)

df.rename({'Takeout option':'Takeaway option'}, axis=1, inplace=True)

clean_data = df
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Region           200 non-null    object  
 1   Place name       200 non-null    object  
 2   Place type       200 non-null    object  
 3   Rating           200 non-null    category
 4   Reviews          200 non-null    float64 
 5   Price            200 non-null    category
 6   Delivery option  200 non-null    bool    
 7   Dine in option   200 non-null    bool    
 8   Takeaway option  200 non-null    bool    
dtypes: bool(3), category(2), float64(1), object(3)
memory usage: 8.1+ KB


# Task 2 

The team at Java June believe that the number of reviews changes depending on the rating. 

Producing a table showing the difference in the median number of reviews by rating along with the minimum and maximum number of reviews to investigate this question for the team.

 - You should start with the data in the file 'coffee.csv'.

 - Your output should be a data frame named `reviews_by_rating`. 

 - It should include the three columns `rating`, `med_review`, `min_review`, `max_review`. 

 - Your answers should be rounded to 1 decimal place.   

In [28]:
# Use this cell to write your code for task 2

import pandas as pd
import numpy as np

df = pd.read_csv('coffee.csv')

reviews_by_rating = df.groupby('Rating')['Reviews'].agg(['median', 'min', 'max']).round(1)
reviews_by_rating.rename(columns={'Rating':'rating', 'median':'med_review', 'min':'min_review', 'max':'max_review'}, inplace=True)
reviews_by_rating

,med_review,min_review,max_review
Rating,,,
3.9,9.5,9.0,10.0
4.0,804.5,170.0,1439.0
4.1,452.5,189.0,716.0
4.2,497.0,385.0,609.0
4.3,221.5,3.0,1656.0
4.4,536.0,40.0,1201.0
4.5,688.0,27.0,2914.0
4.6,693.0,11.0,2931.0
4.7,400.0,80.0,17937.0


# Task 3

Fit a baseline model to predict the number of reviews a store will get.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `Place name` and `rating`. The rating column must be your predicted values.

In [29]:
# Use this cell to write your code for task 3

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Import data
train = pd.read_csv('train.csv')
val = pd.read_csv('validation.csv')

# Fill empty values
dict = {'Rating': 0,
       'Reviews': df['Reviews'].median(),
       'Dine.in.option': False,
       'Takeout.option': False}
dict_1 = {'Rating': 0,
          'Dine.in.option': False,
          'Takeout.option': False}

train.fillna(dict, inplace=True)
val.fillna(dict_1, inplace=True)

# Drop 'Place.name' from train and val data 
train_drop = train.drop(columns=['Place.name'])
val_drop = val.drop(columns=['Place.name'])

# Get dummy values for columns
train_dummies = pd.get_dummies(train_drop, columns=['Region', 'Place.type', 'Price'])
val_dummies = pd.get_dummies(val_drop, columns=['Region', 'Place.type', 'Price'])

# val_dummies is missing the column for Price_$, need to replace
val_dummies['Price_$'] = np.nan
val_dummies['Price_$'].fillna(0, inplace=True)

# Instantiate the linear model
model = LinearRegression()

# Create X and y variables
X = train_dummies.drop('Reviews', axis=1).values
y = train_dummies['Reviews'].values

# Fit the model to the training data
model.fit(X, y)

# Predict values based on val_data
predicted_reviews = model.predict(val_dummies)

# Create dataframe of predicted values
base_result = pd.DataFrame({'Place name':val['Place.name'],
                            'rating':predicted_reviews})
base_result

,Place name,rating
0,Коферум,7.218235
1,Кофейня Starcoff,1037.320075
2,SOVA COFFEE,925.304066
3,Кава Тайм,505.377922
4,Dzhi,2072.625047
5,Koffishka,1618.851414
6,Verona,1854.917588
7,I love coffee,1219.447055
8,Дом Кофе,1247.451058
9,Prostir.coffee,2255.392383


# Task 4

Fit a comparison model to predict the number of reviews a store will get.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `Place name` and `rating`. The rating column must be your predicted values.

In [30]:
# Use this cell to write your code for task 4

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Import data
train = pd.read_csv('train.csv')
val = pd.read_csv('validation.csv')

# Fill empty values
dict = {'Rating': 0,
       'Reviews': df['Reviews'].median(),
       'Dine.in.option': False,
       'Takeout.option': False}
dict_1 = {'Rating': 0,
          'Dine.in.option': False,
          'Takeout.option': False}

train.fillna(dict, inplace=True)
val.fillna(dict_1, inplace=True)

# Drop 'Place.name' from train and val data 
train_drop = train.drop(columns=['Place.name'])
val_drop = val.drop(columns=['Place.name'])

# Get dummy values for columns
train_dummies = pd.get_dummies(train_drop, columns=['Region', 'Place.type', 'Price'])
val_dummies = pd.get_dummies(val_drop, columns=['Region', 'Place.type', 'Price'])

# val_dummies is missing the column for Price_$, need to replace
val_dummies['Price_$'] = np.nan
val_dummies['Price_$'].fillna(0, inplace=True)

# Instantiate the linear model
model = RandomForestRegressor()

# Create X and y variables
X = train_dummies.drop('Reviews', axis=1).values
y = train_dummies['Reviews'].values

# Fit the model to the training data
model.fit(X, y)

# Predict values based on val_data
predicted_reviews = model.predict(val_dummies)

# Create dataframe of predicted values
compare_result = pd.DataFrame({'Place name':val['Place.name'],
                            'rating':predicted_reviews})
compare_result

,Place name,rating
0,Коферум,27.390000
1,Кофейня Starcoff,686.677238
2,SOVA COFFEE,189.615500
3,Кава Тайм,34.779333
4,Dzhi,1028.670000
5,Koffishka,818.057905
6,Verona,1209.413833
7,I love coffee,515.875000
8,Дом Кофе,545.847500
9,Prostir.coffee,1080.850000
